Imports

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split

Read CSV

In [4]:
# Set seed
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Load the dataset
dataset = pd.read_csv("../../datasets/final_dataset.csv")

# First split: train and test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset["Text"], dataset["Label"], test_size=0.2, random_state=42, stratify=dataset["Label"]
)

# Second split: train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

Process Data

In [5]:
# Tokenize the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=100)
X_val = pad_sequences(X_val_seq, maxlen=100)
X_test = pad_sequences(X_test_seq, maxlen=100)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)


Neural Network

In [6]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=64),  

    LSTM(16, activation="tanh", return_sequences=True, 
         kernel_regularizer=l2(0.003), recurrent_dropout=0.5),  # Increased L2 and Dropout
    Dropout(0.6),

    LSTM(8, kernel_regularizer=l2(0.003), recurrent_dropout=0.5, return_sequences=False),
    Dropout(0.6),

    BatchNormalization(),

    Dense(16, activation="relu", kernel_regularizer=l2(0.003)),  # Increased L2
    Dropout(0.7),  # Increased Dropout

    Dense(1, activation="sigmoid")
])

# Compile with a lower learning rate
optimizer = Adam(learning_rate=0.0002)  # Further reduced learning rate
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Early Stopping (More aggressive stopping)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,  # Decreased patience for early stopping
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,  # More aggressive reduction
    patience=1,
    min_lr=0.00005
)

# Train with adjusted dropout, regularization, and patience for early stopping
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - accuracy: 0.5262 - loss: 1.0683 - val_accuracy: 0.6579 - val_loss: 0.9509 - learning_rate: 2.0000e-04
Epoch 2/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.5152 - loss: 1.0002 - val_accuracy: 0.7381 - val_loss: 0.9209 - learning_rate: 2.0000e-04
Epoch 3/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.5674 - loss: 0.9013 - val_accuracy: 0.7550 - val_loss: 0.8910 - learning_rate: 2.0000e-04
Epoch 4/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.6033 - loss: 0.8635 - val_accuracy: 0.8243 - val_loss: 0.8613 - learning_rate: 2.0000e-04
Epoch 5/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.6235 - loss: 0.8037 - val_accuracy: 0.8737 - val_loss: 0.8239 - learning_rate: 2.0000e-04
Epoch 6/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.6619 - loss: 0.7698 - val_accuracy: 0.8937 - val_loss: 0.7774 - learning_rate: 2.0000e-04
Epoch 7/15
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.6902

Results

In [7]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}\n")

val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {val_acc:.4f}")

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9381 - loss: 0.4109
Test accuracy: 0.9433

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9489 - loss: 0.3984
Validation accuracy: 0.9522


Benchmarking

In [8]:
# Load new data
new_data = pd.read_csv("../../datasets/validation_dataset.csv", delimiter=";")

# Tokenize and pad the new data (using the same tokenizer you trained on)
X_new_seq = tokenizer.texts_to_sequences(new_data["Text"])
X_new = pad_sequences(X_new_seq, maxlen=100)  # Ensure maxlen is consistent with your training data

# Make predictions
predictions = model.predict(X_new).flatten()

# Convert predictions to labels based on threshold
labels = ["AI" if pred > 0.5 else "Human" for pred in predictions]

# Create output DataFrame with predictions
output_df = pd.DataFrame({
    "Label": labels,
    "Prediction": predictions
})

# Load the ground truth labels (from the same dataset)
# Since the labels are in the 'Label' column, we'll compare them with predictions.
ground_truth = new_data["Label"]

# Calculate accuracy
accuracy = (output_df["Label"] == ground_truth).mean()

# Print the accuracy
print(f"Accuracy: {accuracy:.4f}")

# Merge predictions with ground truth for comparison
comparison_df = output_df.copy()
comparison_df["Label_actual"] = ground_truth

# Show misclassified samples
misclassified = comparison_df[comparison_df["Label"] != comparison_df["Label_actual"]]
print("\nMisclassified Samples:")
print(misclassified)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step
Accuracy: 0.7375

Misclassified Samples:
    Label  Prediction Label_actual
5   Human    0.124157           AI
6   Human    0.126704           AI
9   Human    0.240882           AI
11  Human    0.411136           AI
12  Human    0.140583           AI
13  Human    0.105954           AI
22  Human    0.146398           AI
24  Human    0.233467           AI
28  Human    0.330795           AI
38  Human    0.467187           AI
39  Human    0.324959           AI
42  Human    0.305147           AI
48     AI    0.749000        Human
51  Human    0.202729           AI
54     AI    0.609120        Human
57  Human    0.281737           AI
59  Human    0.420288           AI
67  Human    0.467878           AI
69  Human    0.363551           AI
75  Human    0.472127           AI
79  Human    0.206749           AI


In [11]:
# Load new dataset for prediction
new_text_data = pd.read_csv("../../datasets/dataset3_inputs.csv", delimiter=";")

# Tokenize and pad the new text data
X_new_seq = tokenizer.texts_to_sequences(new_text_data["Text"])
X_new = pad_sequences(X_new_seq, maxlen=100)  # Ensure maxlen is consistent with training data

# Make predictions
new_predictions = model.predict(X_new).flatten()

# Convert predictions to labels based on threshold
new_labels = ["AI" if pred > 0.5 else "Human" for pred in new_predictions]

# Create output DataFrame with ID and predictions
output_new_df = pd.DataFrame({
    "ID": new_text_data["ID"],
    "Label": new_labels
})

# Save the predictions to a new CSV file
output_new_df.to_csv("results-s1.csv", sep=";", index=False)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
